In [6]:
import os
os.environ["CDF_LIB"] = '/home/yjin/Downloads/cdf38_0-dist-all/cdf38_0-dist/lib'
from spacepy import pycdf
import cv2
from PIL import Image
import numpy as np
import imageio
from tqdm import tqdm
from joblib import Parallel, delayed
import h5py
from matplotlib import pyplot as plt

In [7]:
def video2image(video_path,image_folder,image_type):
    vc = cv2.VideoCapture(video_path)
    length = int(vc.get(cv2.CAP_PROP_FRAME_COUNT))
    digits = len(str(length))
    for i in tqdm(range(length)):
        _, image = vc.read()
        pil_image = Image.fromarray(np.uint8(image[..., ::-1]))
        image = np.array(pil_image)
        imageio.imsave(os.path.join(image_folder, f'{str(i).zfill(digits)}.'+image_type), image)
def convert_video(mask_gtbs_dir,video_gtbs_dir,mask):
    mask_path = os.path.join(mask_gtbs_dir,mask)
    video_path = os.path.join(video_gtbs_dir,mask)
    image_mask_folder = os.path.join(image_gtbs_dir,'.'.join(mask.split('.')[0:2])+'_mask')
    image_video_folder = os.path.join(image_gtbs_dir,'.'.join(mask.split('.')[0:2])+'_image')
    if not mask.endswith('mp4'):
        return
    if not os.path.exists(image_mask_folder):
        os.makedirs(image_mask_folder) 
    if not os.path.exists(image_video_folder):
        os.makedirs(image_video_folder)  
    if len(os.listdir(image_mask_folder))==0:
        video2image(mask_path,image_mask_folder,'jpg')
    if len(os.listdir(image_video_folder))==0:
        video2image(video_path,image_video_folder,'jpg')    
def make_mask_image(video,image_common_dir):
    print(video)
    image_dir = os.path.join(image_common_dir,video+'_image')
    mask_dir = os.path.join(image_common_dir,video+'_mask')
    masked_image_dir = os.path.join(image_common_dir,video+'_masked_image')
    if not os.path.isdir(masked_image_dir):
        os.makedirs(masked_image_dir)
    image_list = [image for image in os.listdir(image_dir)]
    mask_list = [mask for mask in os.listdir(mask_dir)]
    common_list = list(set(image_list).intersection(mask_list))
    for common_file in common_list:
        mask = cv2.imread(os.path.join(mask_dir,common_file))
        kernel = np.ones((3,3),np.uint8)
        image = cv2.imread(os.path.join(image_dir,common_file))
        masked_image = image*(mask>100) + (mask<100)*255
        cv2.imwrite(os.path.join(masked_image_dir,common_file), masked_image)

In [8]:
root_dir = './dataset/h36m/'
images_dir = os.path.join(root_dir,'Images')
masks_dir = os.path.join(root_dir,'Segments')
poses_dir = os.path.join(root_dir,'Poses')
videos_dir = os.path.join(root_dir,'Videos')

In [9]:
# VIDEO to FRAME
# iter through scene
for scene in tqdm(os.listdir(masks_dir)):
    scene_name = scene.split('_')[-1]
    video_scene_dir = os.path.join(videos_dir,'Videos_'+scene_name)
    mask_scene_dir = os.path.join(masks_dir,scene)
    image_scene_dir = os.path.join(images_dir,scene_name)
    if not os.path.isdir(mask_scene_dir):
        continue
    # create scence folder in image folder
    if not os.path.exists(image_scene_dir):
        os.makedirs(image_scene_dir)
    # iter through actor
    for actor in tqdm(os.listdir(mask_scene_dir)):
        if actor=='S10':
            continue
        mask_actor_dir = os.path.join(mask_scene_dir,actor)
        video_actor_dir = os.path.join(video_scene_dir,actor)
        image_actor_dir = os.path.join(image_scene_dir,actor)
        if not os.path.isdir(mask_actor_dir):
            continue
        # create actor folder in image folder
        if not os.path.exists(image_actor_dir):
            os.makedirs(image_actor_dir)
        # skip common subfodler
        mask_common_dir = os.path.join(mask_actor_dir,'MySegmentsMat')
        image_commmon_dir = os.path.join(image_actor_dir,'MyImages')
        video_commmon_dir = os.path.join(video_actor_dir,'Videos')
        if not os.path.exists(image_commmon_dir):
            os.makedirs(image_commmon_dir)  
        # skip gt_bs subfolder
        mask_gtbs_dir = os.path.join(mask_common_dir,'ground_truth_bs')
        image_gtbs_dir = image_commmon_dir
        video_gtbs_dir = video_commmon_dir
        if not os.path.exists(image_gtbs_dir):
            os.makedirs(image_gtbs_dir)  
        # iter through videos
        _ = Parallel(n_jobs=4)(delayed(convert_video)(mask_gtbs_dir,video_gtbs_dir,mask) for mask in tqdm(os.listdir(mask_gtbs_dir)))

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

100%|██████████| 8/8 [01:50<00:00, 13.79s/it]

100%|██████████| 1/1 [04:01<00:00, 241.75s/it]


In [10]:
# Mask Image
# iter through scenemasks_dir
for scene in tqdm(os.listdir(images_dir)):
# for scene in tqdm(['Discussion','Directions']):
    image_scene_dir = os.path.join(images_dir,scene)
    print(scene)
    # iter through actor
    for actor in tqdm(os.listdir(image_scene_dir)):
        print(actor)
        if actor=='S10':
            continue
        image_actor_dir = os.path.join(image_scene_dir,actor)
        image_common_dir = os.path.join(image_actor_dir,'MyImages')
        video_list = list(set([video.split('_')[0] for video in os.listdir(image_common_dir)]))
        _ = Parallel(n_jobs=8)(delayed(make_mask_image)(video,image_common_dir) for video in tqdm(video_list))

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

Directions
S1




100%|██████████| 8/8 [00:00<00:00, 51.55it/s]

100%|██████████| 1/1 [02:06<00:00, 126.05s/it]


In [11]:
img_list_txt = os.path.join(root_dir,'cropped_img_list.txt')
pose_file = os.path.join(root_dir,'cropped_pose.h5')
cropped_images_dir = os.path.join(root_dir,'Cropped_Images')
if os.path.isfile(pose_file):
    os.remove(pose_file)

In [12]:
# crop image
h5_f = h5py.File(pose_file, 'w')
img_f = open(img_list_txt, 'w')
for scene in tqdm(os.listdir(images_dir)):
# for scene in tqdm(['Walking']):
    cropped_images_scene_dir = os.path.join(cropped_images_dir,scene)
    if not os.path.exists(cropped_images_scene_dir):
        os.makedirs(cropped_images_scene_dir)     
    image_scene_dir = os.path.join(images_dir,scene)
    pose_scene_dir = os.path.join(poses_dir,'Poses_D2_Positions_'+scene)
    # iter through actor
    for actor in tqdm(os.listdir(pose_scene_dir)):
#     for actor in tqdm(['S1']):
        cropped_images_actor_dir = os.path.join(cropped_images_scene_dir,actor)
        if not os.path.exists(cropped_images_actor_dir):
            os.makedirs(cropped_images_actor_dir)   
        image_actor_dir = os.path.join(image_scene_dir,actor)
        pose_actor_dir = os.path.join(pose_scene_dir,actor)
        image_common_dir = os.path.join(image_actor_dir,'MyImages')
        pose_common_dir = os.path.join(pose_actor_dir,'MyPoseFeatures')
        d2_pose_dir = os.path.join(pose_common_dir,'D2_Positions')
        video_list = list(set([video.split('_')[0] for video in os.listdir(image_common_dir)]))
        
        for video in tqdm(video_list):          
            pose_data =pycdf.CDF(os.path.join(d2_pose_dir,video+'.cdf'))['Pose'][0]                         
            cropped_image_dir = os.path.join(cropped_images_actor_dir,video) 
            masked_image_dir = os.path.join(image_common_dir,video+"_masked_image")
            if not os.path.exists(cropped_image_dir):
                os.makedirs(cropped_image_dir) 
            masked_image_list = [f for f in os.listdir(masked_image_dir)]
            masked_image_list.sort()
            for idx,file in enumerate(masked_image_list):
                img = cv2.imread(os.path.join(masked_image_dir,file))
                x =(int)((pose_data[idx][8]+pose_data[idx][20])/2)
                y_max =(int)(max(pose_data[idx][9],pose_data[idx][21]))
                y_min =(int)(pose_data[idx][31])
                h = y_max-y_min
                y_mar = int((y_max-y_min)/10)
                w = int((h+2*y_mar)/2)
                cropped_img = img[max(y_min-y_mar,0):min(y_max+y_mar,img.shape[0]),max(0,x-w):min(x+w,img.shape[1]),:]
                
                resized = cv2.resize(cropped_img,(224,224), interpolation = cv2.INTER_AREA)
                cv2.imwrite(os.path.join(cropped_image_dir,file), resized)
                x_offset = max(0,x-w)
                y_offset = max(y_min-y_mar,0)
                sride_x = 224/cropped_img.shape[1]
                sride_y = 224/cropped_img.shape[0]
                for i in range(0,63,2):
                    pose_data[idx][i] = (pose_data[idx][i] - x_offset)*sride_x
                    pose_data[idx][i+1] = (pose_data[idx][i+1] - y_offset)*sride_y
                if np.sum(pose_data[idx]<0)!=0 or np.sum(pose_data[idx]>223)!=0:
                    print('invalid image {}'.format(idx))
                    continue
                img_f.write(os.path.join(cropped_image_dir,file)[len(root_dir):])
                img_f.write('\n')
#                 print(os.path.join(cropped_image_dir,file))
                h5_f.create_dataset(os.path.join(cropped_image_dir,file)[len(root_dir):], data=np.array(pose_data[idx]))
h5_f.close()
img_f.close()

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

invalid image 380
invalid image 381
invalid image 382
invalid image 383
invalid image 384
invalid image 385
invalid image 386
invalid image 387
invalid image 388
invalid image 389
invalid image 390
invalid image 391
invalid image 392
invalid image 393
invalid image 394
invalid image 395
invalid image 396
invalid image 397
invalid image 398
invalid image 399
invalid image 400
invalid image 401
invalid image 402
invalid image 403
invalid image 404
invalid image 405
invalid image 406
invalid image 407
invalid image 408
invalid image 409
invalid image 410
invalid image 411
invalid image 412
invalid image 413
invalid image 414
invalid image 415
invalid image 416
invalid image 417
invalid image 418
invalid image 419
invalid image 420
invalid image 421
invalid image 422
invalid image 423
invalid image 424
invalid image 425
invalid image 426
invalid image 427
invalid image 428
invalid image 429
invalid image 430
invalid image 431
invalid image 432
invalid image 433
invalid image 434
invalid im



 12%|█▎        | 1/8 [00:15<01:46, 15.27s/it]

invalid image 337
invalid image 338
invalid image 339
invalid image 340
invalid image 341
invalid image 342
invalid image 343
invalid image 344
invalid image 345
invalid image 346
invalid image 347
invalid image 348
invalid image 349
invalid image 350
invalid image 351
invalid image 352
invalid image 353
invalid image 354
invalid image 355
invalid image 356
invalid image 357
invalid image 358
invalid image 359
invalid image 360
invalid image 361
invalid image 362
invalid image 363
invalid image 364
invalid image 365
invalid image 366
invalid image 367
invalid image 368
invalid image 369
invalid image 370
invalid image 371
invalid image 372
invalid image 373
invalid image 374
invalid image 375
invalid image 376
invalid image 377
invalid image 378
invalid image 379
invalid image 380
invalid image 381
invalid image 382
invalid image 383
invalid image 384
invalid image 385
invalid image 386
invalid image 387
invalid image 388
invalid image 389
invalid image 390
invalid image 398
invalid im



 25%|██▌       | 2/8 [00:28<01:25, 14.29s/it]

...



 75%|███████▌  | 6/8 [01:25<00:28, 14.23s/it]

invalid image 381
invalid image 382
invalid image 383
invalid image 384
invalid image 385
invalid image 386
invalid image 387
invalid image 388
invalid image 389
invalid image 390
invalid image 391
invalid image 392
invalid image 393
invalid image 394
invalid image 395
invalid image 396
invalid image 397
invalid image 398
invalid image 399
invalid image 400
invalid image 401
invalid image 402
invalid image 403
invalid image 404
invalid image 405
invalid image 406
invalid image 407
invalid image 408
invalid image 409
invalid image 410
invalid image 411
invalid image 412
invalid image 413
invalid image 414
invalid image 415
invalid image 416
invalid image 417
invalid image 418
invalid image 419
invalid image 420
invalid image 421
invalid image 422
invalid image 423
invalid image 424
invalid image 425
invalid image 426
invalid image 427
invalid image 428
invalid image 429
invalid image 430
invalid image 431
invalid image 432
invalid image 433
invalid image 434
invalid image 435
invalid im



 88%|████████▊ | 7/8 [01:40<00:14, 14.43s/it]

invalid image 341
invalid image 342
invalid image 343
invalid image 344
invalid image 345
invalid image 346
invalid image 347
invalid image 348
invalid image 349
invalid image 350
invalid image 351
invalid image 352
invalid image 353
invalid image 354
invalid image 355
invalid image 356
invalid image 357
invalid image 358
invalid image 359
invalid image 360
invalid image 361
invalid image 362
invalid image 368
invalid image 382
invalid image 383
invalid image 384
invalid image 385
invalid image 386
invalid image 387
invalid image 388
invalid image 389
invalid image 390
invalid image 398
invalid image 399
invalid image 400
invalid image 401
invalid image 402
invalid image 403
invalid image 404
invalid image 405
invalid image 406
invalid image 407
invalid image 408
invalid image 566
invalid image 567
invalid image 568
invalid image 569
invalid image 570
invalid image 571
invalid image 572
invalid image 573
invalid image 574
invalid image 575
invalid image 576
invalid image 577
invalid im



100%|██████████| 8/8 [01:53<00:00, 14.15s/it]

100%|██████████| 1/1 [01:53<00:00, 113.21s/it]


In [27]:
# create train test split
full_list_txt = './dataset/h36m/cropped_img_list.txt'
train_list_txt = './dataset/h36m/train_img_list.txt'
test_list_txt = './dataset/h36m/test_img_list.txt'
with open(full_list_txt, 'r') as full_f , \
     open(train_list_txt, 'w') as train_f , \
     open(test_list_txt, 'w') as test_f:
    lines = full_f.readlines() 
    for line in lines:
        if 'S1' in line:
            test_f.write(line)
        else:
            train_f.write(line)